# Import Libary Package

In [13]:
!pip install tqdm

In [14]:
import pandas as pd
import numpy as np
import json
import re
from difflib import SequenceMatcher
import itertools
from itertools import combinations, product, chain
from pprint import pprint
from ast import literal_eval
from tqdm import tqdm

from __future__ import unicode_literals
import operator
import sys

try:
    from html.parser import HTMLParser
except ImportError:
    from HTMLParser import HTMLParser

try:
    import html
except ImportError:
    pass

# Load data

In [22]:
path = '/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/downstream_data.csv'
data = pd.read_csv(path , converters= {'input': literal_eval})
data = data.dropna()
data

,dataset,type,sent_idx,idx,original,input
0,ViHSD,train,0,0,Em được làm fan cứng luôn rồi nè ❤ ️ reaction ...,"[Em, được, làm, fan, cứng, luôn, rồi, nè, ❤, ️..."
1,ViHSD,train,1,0,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,"[Đúng, là, bọn, mắt, híp, lò xo, thụt, :))), b..."
2,ViHSD,train,2,0,@username giờ giống thằng sida hơn à,"[@username, giờ, giống, thằng, sida, hơn, à]"
3,ViHSD,train,3,0,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN . NHÀ N...,"[CÔN ĐỒ, CỤC SÚC, VÔ NHÂN, TÍNH, ĐỀ, NGHI, VN,..."
4,ViHSD,train,4,0,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,"[Từ, lý thuyết, đến, thực hành, là, cả, 1, câu..."
...,...,...,...,...,...,...
121081,ViSpamReviews,test,19866,2,Dùng rải lớp nhẹ trên mặt chậu lan hoặc làm ch...,"[Dùng, rải, lớp, nhẹ, trên, mặt, chậu, lan, ho..."
121083,ViSpamReviews,test,19866,4,Rêu có khả năng giữ nước và cho rễ cây bám hút...,"[Rêu, có, khả năng, giữ, nước, và, cho, rễ, câ..."
121084,ViSpamReviews,test,19867,0,Shop giao hàng nhanh nhưng mỗi tội cái shopee ...,"[Shop, giao, hàng, nhanh, nhưng, mỗi tội, cái,..."
121085,ViSpamReviews,test,19868,0,Shop ơi em có đặt 2 ram 8g dr4 nhưng máy em nh...,"[Shop, ơi, em, có, đặt, 2, ram, 8g, dr4, nhưng..."


In [23]:
# prompt: Keep first 500 row of each dataset in data

data = data.groupby('dataset').head(500)
data = data.reset_index(drop=False)
data

,index,dataset,type,sent_idx,idx,original,input
0,0,ViHSD,train,0,0,Em được làm fan cứng luôn rồi nè ❤ ️ reaction ...,"[Em, được, làm, fan, cứng, luôn, rồi, nè, ❤, ️..."
1,1,ViHSD,train,1,0,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,"[Đúng, là, bọn, mắt, híp, lò xo, thụt, :))), b..."
2,2,ViHSD,train,2,0,@username giờ giống thằng sida hơn à,"[@username, giờ, giống, thằng, sida, hơn, à]"
3,3,ViHSD,train,3,0,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN . NHÀ N...,"[CÔN ĐỒ, CỤC SÚC, VÔ NHÂN, TÍNH, ĐỀ, NGHI, VN,..."
4,4,ViHSD,train,4,0,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,"[Từ, lý thuyết, đến, thực hành, là, cả, 1, câu..."
...,...,...,...,...,...,...,...
2495,83115,ViSpamReviews,train,271,0,Ban đầu mình hay dùng dòng Skin aqua - milk .,"[Ban đầu, mình, hay, dùng, dòng, Skin, aqua, -..."
2496,83116,ViSpamReviews,train,271,1,Đến khi có dòng tone up này thì chuyển sang xà...,"[Đến, khi, có, dòng, tone, up, này, thì, chuyể..."
2497,83117,ViSpamReviews,train,271,2,Vẫn rất ưng ý .,"[Vẫn, rất, ưng ý, .]"
2498,83118,ViSpamReviews,train,271,3,Dòng sản phẩm này rất hợp với da dầu mụn như m...,"[Dòng, sản phẩm, này, rất, hợp, với, da dầu, m..."


In [46]:
word_normalized = pd.DataFrame()
for i in tqdm (range(len(data)), desc="Loading..."):
  input = data.input[i].copy()
  x = pd.DataFrame(input, columns=['token'])
  sentence = "Index: " + str(data.index[i])
  word_normalized = pd.concat([word_normalized,pd.DataFrame(data = {'index': [sentence], 0: np.nan})])
  word_normalized = pd.concat([word_normalized, x.reset_index(drop=False)])

word_normalized.columns = ['index', 'token_idx', 'token']
word_normalized['token_idx'] = word_normalized['index']

sent_idx = -1
for row in range(len(word_normalized)):
    if('Index' in str(word_normalized.iloc[row, 0])):
      word_normalized.iloc[row, 0] = np.nan
      word_normalized.iloc[row, 1] = np.nan
      sent_idx += 1
    else:
      word_normalized.iloc[row, 0] = sent_idx

word_normalized = word_normalized.dropna(subset = ['index','token_idx', 'token'], how='all')
word_normalized = word_normalized.reset_index(drop=True)
word_normalized['token_idx'] = word_normalized['token_idx'].astype(int)

Loading...: 100%|██████████| 2500/2500 [00:11<00:00, 220.84it/s]


In [47]:
word_normalized

,index,token_idx,token
0,0,0,Em
1,0,1,được
2,0,2,làm
3,0,3,fan
4,0,4,cứng
...,...,...,...
30813,2499,13,lót
30814,2499,14,","
30815,2499,15,kem
30816,2499,16,nền


In [48]:
#word_normalized.to_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/Annotation/unlabeled_500.csv')